In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from scipy.signal import hilbert
from antropy import sample_entropy


In [13]:
df = pd.read_csv("C:/Users/lazar/Downloads/S002_PSG_df_updated.csv")

signal = df['F4-M1'].values

fs = 100 
epoch_length = 30  
samples_per_epoch = fs * epoch_length 

num_epochs = len(signal) // samples_per_epoch
epohe = np.array_split(signal[:num_epochs * samples_per_epoch], num_epochs)

In [14]:
epohe[0].size

3000

In [15]:
bandovi = {
    "delta": (0.5, 4),
    "theta": (4, 8),
    "alpha": (8, 13),
    "beta": (13, 30),
    "gamma": (30, 45)
}

def bandpower(sig, fs, band):  #filtrira signale po frekvencijama
    low, high = band
    b, a = butter(4, [low / (fs / 2), high / (fs / 2)], btype='band')
    filtered = filtfilt(b, a, sig)
    power = np.sum(filtered ** 2) / len(filtered)
    return power 

In [16]:
epohe_features = []

for epoha in epohe:
    # RACUNANJE FEATUREA
    rms = np.sqrt(np.mean(np.square(epoha))) #ROOT MEAN SQUARE
    mse = np.mean(np.square(epoha)) #MEAN SQUARED ERROR
    mmd = np.max(epoha) - np.min(epoha) #MINIMAL MAXIMAL DISTANCE
    zcr = ((epoha[:-1] * epoha[1:]) < 0).sum() #ZERO CROSSING RATE
    se = sample_entropy(epoha) #SAMPLE ENTROPY

    talasi = {ime: bandpower(epoha, fs, opseg) for ime, opseg in bandovi.items()}

    red = {
        "RMS": rms,
        "MSE": mse,
        "MMD": mmd,
        "ZCR": zcr,
        "Sample Entropy": se,
        "Delta": talasi["delta"],
        "Theta": talasi["theta"],
        "Alpha": talasi["alpha"],
        "Beta": talasi["beta"],
        "Gamma": talasi["gamma"]
    }

    epohe_features.append(red)


In [18]:
epohe_features[0]

{'RMS': np.float64(1.952208921289194e-06),
 'MSE': np.float64(3.811119672361119e-12),
 'MMD': np.float64(1.347676813916228e-05),
 'ZCR': np.int64(224),
 'Sample Entropy': 0.8002773336676623,
 'Delta': np.float64(2.1453640729610608e-12),
 'Theta': np.float64(3.72501464766949e-13),
 'Alpha': np.float64(8.436520758288404e-14),
 'Beta': np.float64(4.9607951237039713e-14),
 'Gamma': np.float64(2.9346405922494765e-15)}

In [ ]:
data_za_korelaciju = pd.DataFrame(epohe_features) #pravljenje dataframea
corr_matrix = data_za_korelaciju.corr() #racunanje korelacije parametara

corr_matrix

,RMS,MSE,MMD,ZCR,Sample Entropy,Delta,Theta,Alpha,Beta,Gamma
RMS,1.000000,0.917091,0.965791,-0.181539,-0.334621,0.919763,0.701656,0.747025,0.677378,0.670982
MSE,0.917091,1.000000,0.806729,-0.106428,-0.177218,0.983698,0.909083,0.932524,0.894306,0.890428
MMD,0.965791,0.806729,1.000000,-0.164480,-0.358995,0.819649,0.520464,0.579778,0.490585,0.482714
ZCR,-0.181539,-0.106428,-0.164480,1.000000,0.768556,-0.113002,-0.061057,-0.063327,-0.055218,-0.054727
Sample Entropy,-0.334621,-0.177218,-0.358995,0.768556,1.000000,-0.180889,-0.098617,-0.113148,-0.089609,-0.087417
Delta,0.919763,0.983698,0.819649,-0.113002,-0.180889,1.000000,0.863547,0.905992,0.847476,0.842229
Theta,0.701656,0.909083,0.520464,-0.061057,-0.098617,0.863547,1.000000,0.990156,0.999279,0.998838
Alpha,0.747025,0.932524,0.579778,-0.063327,-0.113148,0.905992,0.990156,1.000000,0.986660,0.984537
Beta,0.677378,0.894306,0.490585,-0.055218,-0.089609,0.847476,0.999279,0.986660,1.000000,0.999915
Gamma,0.670982,0.890428,0.482714,-0.054727,-0.087417,0.842229,0.998838,0.984537,0.999915,1.000000


In [49]:

for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        col1 = corr_matrix.columns[i]
        col2 = corr_matrix.columns[j]
        corr_value = corr_matrix.iloc[i, j]
        if abs(corr_value) > 0.9:
            print(f"{col1} i {col2}: {corr_value:.2f}")

RMS i MSE: 0.92
RMS i MMD: 0.97
RMS i Delta: 0.92
MSE i Delta: 0.98
MSE i Theta: 0.91
MSE i Alpha: 0.93
Delta i Alpha: 0.91
Theta i Alpha: 0.99
Theta i Beta: 1.00
Theta i Gamma: 1.00
Alpha i Beta: 0.99
Alpha i Gamma: 0.98
Beta i Gamma: 1.00
